In [ ]:
import xarray as xr

ds = xr.open_dataset("E/E_2003_GLEAM_v4.2b_MO.nc")
print(ds)

E = ds["E"]
E_global = E.mean(dim=["lat", "lon"])

print(E_global.values)

<xarray.Dataset> Size: 311MB
Dimensions:  (time: 12, lat: 1800, lon: 3600)
Coordinates:
  * time     (time) datetime64[ns] 96B 2003-01-31 2003-02-28 ... 2003-12-31
  * lat      (lat) float64 14kB 89.95 89.85 89.75 89.65 ... -89.75 -89.85 -89.95
  * lon      (lon) float64 29kB -179.9 -179.8 -179.8 ... 179.7 179.8 179.9
Data variables:
    E        (time, lat, lon) float32 311MB ...
Attributes:
    Dataset:      Global Land Evaporation Amsterdam Model
    Version:      4.2b
    Authors:      Hydro-Climate Extremes Lab (H-CEL)
    Institution:  Ghent University
    Contact:      info@gleam.eu
    Reference1:   Miralles, D.G. et al. 2011: Global land-surface evaporation...
    Reference2:   Miralles, D. G., Bonte, O., Koppa, A., Baez-Villanueva, O. ...
[22.161547 21.372425 25.728031 30.65403  37.179916 41.664604 43.680386
 38.833107 30.867937 26.287262 23.234268 23.04364 ]


In [ ]:
import xarray as xr
import pandas as pd
import glob
import os

data_dir = "E"
nc_files = sorted(glob.glob(os.path.join(data_dir, "E_*_GLEAM_v4.2b_MO.nc")))

results = []

for file in nc_files:
    # 从文件名中提取年份
    # E_2003_GLEAM_v4.2b_MO.nc → 2003
    year = int(os.path.basename(file).split("_")[1])

    ds = xr.open_dataset(file)

    # 变量 E
    E = ds["E"]

    # 空间平均（lat, lon）
    E_global = E.mean(dim=["lat", "lon"])

    # time → month
    months = E_global["time"].dt.month.values
    values = E_global.values

    # 存结果
    for m, v in zip(months, values):
        results.append({
            "year": year,
            "month": int(m),
            "evaporation": float(v)
        })

    ds.close()


df = pd.DataFrame(results)

df = df.sort_values(["year", "month"]).reset_index(drop=True)


df.to_csv("GLEAM_global_monthly_evaporation.csv", index=False)

print("Saved to GLEAM_global_monthly_evaporation.csv")
print(df.head())


Saved to GLEAM_global_monthly_evaporation.csv
   year  month  evaporation
0  2003      1    22.161547
1  2003      2    21.372425
2  2003      3    25.728031
3  2003      4    30.654030
4  2003      5    37.179916
